# Food For Thought 
## Digging into the dataset

The first step of this project aims at understanding the dataset we have chosen ([Open Food Facts Database](https://world.openfoodfacts.org/)), to check whether it is suitable for the kind of analysis we want to develop.

As a reminder, we would like to focus our research on two main topics:
1. Impact of food on environment
2. Impact of food on the health

In [1]:
import re
import pandas as pd
import numpy as np
import matplotlib
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.style as style
from mpl_toolkits.mplot3d import Axes3D
from difflib import get_close_matches

import pickle

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

import findspark
findspark.init()

from pyspark.sql import *
import pyspark.sql.functions as F

from pyspark.sql import SparkSession
from pyspark import SparkContext

spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext
%matplotlib inline

In [2]:
from plotly.offline import download_plotlyjs, init_notebook_mode
import plotly.plotly as py
import plotly.graph_objs as go
import plotly.tools as tools

#import cufflinks as cf 

init_notebook_mode(connected=True)

DATA_FOLDER = './data/'
data = spark.read.csv(DATA_FOLDER +'en.openfoodfacts.org.products.csv',header=True,sep='\t')          

### Brand vs Nutrition vs Food Category in France 

#### load and clean

bra_cat_nut = data.filter("countries_en=='France'")\
                    .select('brands', 'nutrition_grade_fr', 'pnns_groups_1')\
                    .dropna(how='all')

bra_cat_nut = bra_cat_nut.withColumn('pnns_groups_1',(F.regexp_replace('pnns_groups_1', '-', ' ')))
square_udf_int = F.udf(lambda z: z.title() if z else z)
bra_cat_nut = bra_cat_nut.withColumn('pnns_groups_1',square_udf_int('pnns_groups_1'))

bra_cat_nut.show(5)

In [ ]:
# we can now read it everytime we want to continue the analysis
bra_cat_nut_raw_df = pd.read_pickle('bra_cat_nut_raw_df')

In [ ]:
bra_cat_nut_raw_df.head()

The problem is that the 'brands' tags are not always consistent. For example we can find the brand tag 'carrefour', "Carrefour", "carrfour,bio carrefour' ect. 
Hence we first clean the brands using a list of food brands present in France and a function to match the 'brands' tag with the closest brand in the list. 

In [ ]:
# list of food brands present in France ('Coca Cola', 'Fleury Michon', 'Carrefour' ect.)
list_brand_file = open("brand_list.txt", "r")
list_brand = list_brand_file.read().split('\n')

# dictonary of parent brands. 
# to replace brands by its parent brand when they belong to a bigger one 
# (eg: 'Bio Village' is a brand of Leclerc which owes also 'Marque Repère')
dict_parent_brand= {'Bio Village': 'Leclerc',
                    'Marque Repère': 'Leclerc'}

# we set to None the brand when it is "Sans Marque" 
# "Sans Marque" means the brands had not been specified, it can be "Marque Repère", "Carrefour", "Délisse" ect. 
# using the product search tool on https://fr.openfoodfacts.org with the tag "Sans Marque".
dict_parent_brand['Sans Marque']= None

In [ ]:
# Function which find for each value (string) in the column 'brands' the closest brand (string) in 'list_brand'.
# Note: the function 'get_close_matches' has been imported from the open-source 'difflib' library.
def clean_brand(brand_name):
    
    # brand name can be of the form 'Carrefour' (one word) or 'Carrefour, Bio Carrefour' (multiple words)
    # hence we split the string w.r.t the symbol ',' and find the best matching brand word for each split.
    matches=[]
    if brand_name:
        
        for word in brand_name.split(','):

            # the cutoff control for the False Positives possibilities in the 'list_brand' 
            # that don’t score at least the cutoff are ignored.
            # n controls for the the number of possibilities (whose score is higher than the cutoff value).
            matches.append(get_close_matches(word.lower(), list_brand, n=1, cutoff=0.6))

        # remove empty sublists and unravel
        matches = [brand for sublist in matches for brand in sublist if sublist]
        
        # output brand
        if matches:
            output = matches[0]
        else:
            output = None
                
        # check for parent brand
        if output in dict_parent_brand:
            output = dict_parent_brand[output]
        
        return output

In [ ]:
# we create a new column with the new "consistent" brand tag. 
# takes some minutes to run...
bra_cat_nut_raw_df['new_brands'] = bra_cat_nut_raw_df['brands'].apply(lambda x: clean_brand(x))

In [ ]:
# drop when both 'nutrition_grade_fr' and 'brands' are None
bra_cat_nut_cleaned_df = bra_cat_nut_raw_df.dropna(subset=['nutrition_grade_fr', 'brands'], how='all') 

# we can drop the old column 'brands' since we have the new ones.
bra_cat_nut_cleaned_df = bra_cat_nut_cleaned_df.drop('brands',1).rename(columns={'new_brands':'brands'})

bra_cat_nut_cleaned_df.head()

In [ ]:
# here we count the number of products for each brand. 
# we consider that brand have enough products to be analyzed from 100 products
count = bra_cat_nut_cleaned_df.groupby('brands').count().sort_values('nutrition_grade_fr', ascending=False)
count.head()

In [ ]:
count['enough_products']= count[['nutrition_grade_fr']].apply(lambda x: x>100)
count = count[count.enough_products]
count.head()

The most abundant brands are Carrefour, Auchan and U. There are the brands of the biggest distributors (supermarkets) in France. 

In [ ]:
# here we add the column "enough products" to the dataframe with the cleaned brand.
bra_cat_nut_cleaned_df = bra_cat_nut_cleaned_df.join(count[['enough_products']], on='brands')
bra_cat_nut_cleaned_df = bra_cat_nut_cleaned_df.drop('enough_products',1)
bra_cat_nut_cleaned_df.head()

In [ ]:
# here we create dummy variables from the 'nutrition_grade_fr' whose value can be:
# - 'a' (very good product for health) 
# - 'b' (good product)
# - 'c' ('neutral product')
# - 'd' ('not so good product')
# - 'e' (bad product for health)

bra_cat_nut_cleaned_expanded_df = pd.get_dummies(bra_cat_nut_cleaned_df.set_index('brands')).reset_index()
bra_cat_nut_cleaned_dummies_df = bra_cat_nut_cleaned_expanded_df.rename(columns=
                              {'nutrition_grade_fr_a':'a',
                               'nutrition_grade_fr_b':'b',
                               'nutrition_grade_fr_c':'c',
                               'nutrition_grade_fr_d':'d',
                               'nutrition_grade_fr_e':'e',
                               'pnns_groups_1_Beverages': 'Beverages',
                               'pnns_groups_1_Cereals And Potatoes':'Cereals And Potatoes',
                               'pnns_groups_1_Composite Foods': 'Composite Foods',
                               'pnns_groups_1_Fat And Sauces':'Fat And Sauces',
                               'pnns_groups_1_Fish Meat Eggs':'Fish Meat Eggs',
                               'pnns_groups_1_Fruits And Vegetables':'Fruits And Vegetables',
                               'pnns_groups_1_Milk And Dairy Products':'Milk And Dairy Products',
                               'pnns_groups_1_Salty Snacks':'Salty Snacks',
                               'pnns_groups_1_Sugary Snacks':'Sugary Snacks',
                               'pnns_groups_1_Unknown': 'Unknown'})
bra_cat_nut_cleaned_dummies_df.head(5)

#### Load Cleaned data

In [4]:
bra_cat_nut_df = pd.read_pickle('bra_cat_nut_cleaned_df')
bra_cat_nut_dummies_df = pd.read_pickle('bra_cat_nut_cleaned_dummies_df')

#### Food quality in the whole dataset

In [5]:
count = bra_cat_nut_df[['nutrition_grade_fr','brands']].groupby('nutrition_grade_fr').count().rename(columns={'brands':'count'})

In [43]:
data = []
colors = sns.color_palette("RdBu_r", 5).as_hex()

for index, row in count.reset_index().iterrows():
    data.append(go.Bar(x=np.array(index),
                       y=np.array(row['count']),
                       name = row['nutrition_grade_fr'],
                       marker={'color': colors[index]}))

layout = go.Layout(
    title='Nutrition score occurences in the original dataset.',
    xaxis=go.layout.XAxis(
        title='Nutriscore',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        ),
        tickvals=np.arange(5).astype(str),
        ticktext=count.index,
    ),
    yaxis=dict(
        title='Count',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    ),
)

go.FigureWidget(data=data, layout=layout)

FigureWidget({
    'data': [{'marker': {'color': '#3783bb'},
              'name': 'a',
              'type': 'bar',
              'uid': '38af53a8-fc96-11e8-a80b-ac220bb8d9d3',
              'x': array([0]),
              'y': array([8951])},
             {'marker': {'color': '#a7d0e4'},
              'name': 'b',
              'type': 'bar',
              'uid': '38af53a9-fc96-11e8-900f-ac220bb8d9d3',
              'x': array([1]),
              'y': array([8831])},
             {'marker': {'color': '#f7f6f6'},
              'name': 'c',
              'type': 'bar',
              'uid': '38af53aa-fc96-11e8-af82-ac220bb8d9d3',
              'x': array([2]),
              'y': array([12866])},
             {'marker': {'color': '#f7b799'},
              'name': 'd',
              'type': 'bar',
              'uid': '38af53ab-fc96-11e8-bcae-ac220bb8d9d3',
              'x': array([3]),
              'y': array([17081])},
             {'marker': {'color': '#c94741'},
              'name': 'e',
              'type': 'bar',
              'uid': '38af53ac-fc96-11e8-8fa5-ac220bb8d9d3',
              'x': array([4]),
              'y': array([13428])}],
    'layout': {'title': 'Nutrition score occurences in the original dataset.',
               'xaxis': {'ticktext': array(['a', 'b', 'c', 'd', 'e'], dtype=object),
                         'tickvals': array(['0', '1', '2', '3', '4'], dtype=object),
                         'title': 'Nutriscore',
                         'titlefont': {'color': '#7f7f7f', 'family': 'Courier New, monospace', 'size': 18}},
               'yaxis': {'title': 'Count',
                         'titlefont': {'color': '#7f7f7f', 'family': 'Courier New, monospace', 'size': 18}}}
})

In [44]:
labels = count.index
values = count['count'].values
colors = sns.color_palette("RdBu_r", 5).as_hex()

In [67]:
fig = {
    'data': [{'labels': labels,
              'values': values,
              'type': 'pie',
              'marker':{'colors':colors}
              }
            ],
    'layout': {'title': 'Nutrition score occurences in the original dataset.'}
     }

py.iplot(fig)

### Food quality for each brand

In [68]:
bra_nut_df = bra_cat_nut_df[['brands', 'nutrition_grade_fr']]
bra_nut_dummies_df = bra_cat_nut_dummies_df.iloc[:,np.arange(6)]

In [69]:
# extract the total number of products for each brands
brand_count = bra_nut_df.groupby('brands').count().rename(columns={'nutrition_grade_fr':'total'})

In [70]:
# here we count the nutrition grade occurences for each brand
brand_dummies_count = bra_nut_dummies_df.groupby('brands').sum()

In [71]:
# we add the column total (with the total number of products sold for each brand) to further extract the ratio of each grade instead of the raw count.
brand_dummies_count = brand_dummies_count.join(brand_count)#.rename(columns = {'nutrition_grade_fr':'total'})
brand_dummies_count.head()

,a,b,c,d,e,total
brands,,,,,,
123 bio,121.0,75.0,54.0,91.0,80.0,421
7Up,0.0,5.0,1.0,4.0,24.0,34
A l'olivier,0.0,2.0,19.0,43.0,3.0,67
Ajax,1.0,0.0,0.0,0.0,1.0,2
Albert Menes,2.0,6.0,27.0,37.0,22.0,94


In [72]:
# we convert the counts into ratios. (ex: ratio for 'a' = count for 'a' / total number of products)
ratio = brand_dummies_count.copy()
ratio[['a','b','c','d','e']] = brand_dummies_count[['a','b','c','d','e']].div(brand_dummies_count['total'].values,axis=0)*100
ratio = ratio.sort_values('total',ascending=False)

In [73]:
ratio_to_plot = ratio.reset_index().loc[:9].drop('total',1)
ratio_to_plot.head()

,brands,a,b,c,d,e
0,Carrefour,19.763931,15.344496,19.818831,26.269558,18.803184
1,Auchan,18.391764,15.303283,21.313300,26.794658,18.196995
2,Leclerc,13.648180,15.901213,22.010399,27.816291,20.623917
3,Casino,18.849840,15.609311,20.903697,25.604747,19.032405
4,Leader Price,17.482517,13.939394,22.750583,26.293706,19.533800


In [74]:
data = []
colors = sns.color_palette("tab10", ratio_to_plot.brands.count()).as_hex()

for index, row in ratio_to_plot.iterrows():
    data.append(go.Bar(x=['a','b','c','d','e'],
                       y=np.array(row[1:]),
                       name = row['brands'],
                       marker={'color': colors[index]}))

layout = go.Layout(
    title='Nutrition score occurences for the 10 biggest food brands in France.',
    xaxis=go.layout.XAxis(
        title='Nutriscore',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        ),
        tickvals=np.arange(5).astype(str),
        ticktext=count.index,
    ),
    yaxis=dict(
        title='Ratio (in %)',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    ),
)

go.FigureWidget(data=data, layout=layout)

FigureWidget({
    'data': [{'marker': {'color': '#1f77b4'},
              'name': 'Carrefour',
              'type': 'bar',
              'uid': 'c332aa76-fc96-11e8-8179-ac220bb8d9d3',
              'x': [a, b, c, d, e],
              'y': array([19.763930826242106, 15.34449629426297, 19.81883063409278,
                          26.269558056546806, 18.80318418885534], dtype=object)},
             {'marker': {'color': '#ff7f0e'},
              'name': 'Auchan',
              'type': 'bar',
              'uid': 'c332aa77-fc96-11e8-b215-ac220bb8d9d3',
              'x': [a, b, c, d, e],
              'y': array([18.39176405119644, 15.30328324986088, 21.313299944351698,
                          26.79465776293823, 18.196994991652755], dtype=object)},
             {'marker': {'color': '#2ca02c'},
              'name': 'Leclerc',
              'type': 'bar',
              'uid': 'c332aa78-fc96-11e8-b27d-ac220bb8d9d3',
              'x': [a, b, c, d, e],
              'y': array([13.648180242634314, 15.901213171577123, 22.0103986135182,
                          27.81629116117851, 20.623916811091856], dtype=object)},
             {'marker': {'color': '#d62728'},
              'name': 'Casino',
              'type': 'bar',
              'uid': 'c332aa79-fc96-11e8-ac3d-ac220bb8d9d3',
              'x': [a, b, c, d, e],
              'y': array([18.849840255591054, 15.60931081697855, 20.903696942035598,
                          25.60474669100867, 19.032405294386127], dtype=object)},
             {'marker': {'color': '#9467bd'},
              'name': 'Leader Price',
              'type': 'bar',
              'uid': 'c332aa7a-fc96-11e8-980e-ac220bb8d9d3',
              'x': [a, b, c, d, e],
              'y': array([17.482517482517483, 13.939393939393941, 22.75058275058275,
                          26.293706293706293, 19.533799533799534], dtype=object)},
             {'marker': {'color': '#8c564b'},
              'name': 'Picard',
              'type': 'bar',
              'uid': 'c332aa7b-fc96-11e8-9815-ac220bb8d9d3',
              'x': [a, b, c, d, e],
              'y': array([29.741650913673602, 18.147448015122876, 21.865154379332072,
                          23.566477630749844, 6.679269061121613], dtype=object)},
             {'marker': {'color': '#e377c2'},
              'name': 'Cora',
              'type': 'bar',
              'uid': 'c332aa7c-fc96-11e8-a48d-ac220bb8d9d3',
              'x': [a, b, c, d, e],
              'y': array([15.635858234885337, 13.273106323835998, 21.195274496177902,
                          27.72758860319666, 22.1681723419041], dtype=object)},
             {'marker': {'color': '#7f7f7f'},
              'name': 'Monoprix',
              'type': 'bar',
              'uid': 'c332aa7d-fc96-11e8-9fe5-ac220bb8d9d3',
              'x': [a, b, c, d, e],
              'y': array([20.1294033069734, 16.031631919482386, 21.49532710280374,
                          26.168224299065418, 16.175413371675056], dtype=object)},
             {'marker': {'color': '#bcbd22'},
              'name': 'Belle France',
              'type': 'bar',
              'uid': 'c332aa7e-fc96-11e8-a2c2-ac220bb8d9d3',
              'x': [a, b, c, d, e],
              'y': array([16.042345276872965, 12.703583061889251, 21.009771986970684,
                          30.374592833876225, 19.86970684039088], dtype=object)},
             {'marker': {'color': '#17becf'},
              'name': 'Netto',
              'type': 'bar',
              'uid': 'c332aa7f-fc96-11e8-bc51-ac220bb8d9d3',
              'x': [a, b, c, d, e],
              'y': array([10.146699266503667, 13.447432762836186, 20.904645476772615,
                          33.12958435207824, 22.371638141809292], dtype=object)}],
    'layout': {'title': 'Nutrition score occurences for the 10 biggest food brands in France.',
               'xaxis': {'ticktext': array(['Beverages', 'Cereals And Potatoes', 'Composite Foods',
                                            'Fat And Sauces', 'F

In [75]:
data = []
colors = sns.color_palette("RdBu_r", 5).as_hex()

for index, nutri in enumerate(['a','b','c','d','e']):
    data.append(go.Bar(x=ratio_to_plot['brands'],
                       y=ratio_to_plot[ratio_to_plot.columns[index+1]],
                       name = ratio_to_plot.columns[index+1],
                       marker={'color': colors[index]}))

layout = go.Layout(
    title='Nutrition score distribution within the 10 biggest food brands in France.',
    barmode='stack',
    xaxis=go.layout.XAxis(
        title='Food Brand',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        ),
        tickvals=np.arange(10).astype(str),
        ticktext=ratio_to_plot['brands'],
    ),
    yaxis=dict(
        title='Ratio (in %)',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    ),
)

go.FigureWidget(data=data, layout=layout)

FigureWidget({
    'data': [{'marker': {'color': '#3783bb'},
              'name': 'a',
              'type': 'bar',
              'uid': 'c3861b42-fc96-11e8-b032-ac220bb8d9d3',
              'x': array(['Carrefour', 'Auchan', 'Leclerc', 'Casino', 'Leader Price', 'Picard',
                          'Cora', 'Monoprix', 'Belle France', 'Netto'], dtype=object),
              'y': array([19.76393083, 18.39176405, 13.64818024, 18.84984026, 17.48251748,
                          29.74165091, 15.63585823, 20.12940331, 16.04234528, 10.14669927])},
             {'marker': {'color': '#a7d0e4'},
              'name': 'b',
              'type': 'bar',
              'uid': 'c3861b43-fc96-11e8-89fe-ac220bb8d9d3',
              'x': array(['Carrefour', 'Auchan', 'Leclerc', 'Casino', 'Leader Price', 'Picard',
                          'Cora', 'Monoprix', 'Belle France', 'Netto'], dtype=object),
              'y': array([15.34449629, 15.30328325, 15.90121317, 15.60931082, 13.93939394,
                          18.14744802, 13.27310632, 16.03163192, 12.70358306, 13.44743276])},
             {'marker': {'color': '#f7f6f6'},
              'name': 'c',
              'type': 'bar',
              'uid': 'c3861b44-fc96-11e8-9b8b-ac220bb8d9d3',
              'x': array(['Carrefour', 'Auchan', 'Leclerc', 'Casino', 'Leader Price', 'Picard',
                          'Cora', 'Monoprix', 'Belle France', 'Netto'], dtype=object),
              'y': array([19.81883063, 21.31329994, 22.01039861, 20.90369694, 22.75058275,
                          21.86515438, 21.1952745 , 21.4953271 , 21.00977199, 20.90464548])},
             {'marker': {'color': '#f7b799'},
              'name': 'd',
              'type': 'bar',
              'uid': 'c3861b45-fc96-11e8-bf45-ac220bb8d9d3',
              'x': array(['Carrefour', 'Auchan', 'Leclerc', 'Casino', 'Leader Price', 'Picard',
                          'Cora', 'Monoprix', 'Belle France', 'Netto'], dtype=object),
              'y': array([26.26955806, 26.79465776, 27.81629116, 25.60474669, 26.29370629,
                          23.56647763, 27.7275886 , 26.1682243 , 30.37459283, 33.12958435])},
             {'marker': {'color': '#c94741'},
              'name': 'e',
              'type': 'bar',
              'uid': 'c3861b46-fc96-11e8-ac65-ac220bb8d9d3',
              'x': array(['Carrefour', 'Auchan', 'Leclerc', 'Casino', 'Leader Price', 'Picard',
                          'Cora', 'Monoprix', 'Belle France', 'Netto'], dtype=object),
              'y': array([18.80318419, 18.19699499, 20.62391681, 19.03240529, 19.53379953,
                           6.67926906, 22.16817234, 16.17541337, 19.86970684, 22.37163814])}],
    'layout': {'barmode': 'stack',
               'title': 'Nutrition score distribution within the 10 biggest food brands in France.',
               'xaxis': {'ticktext': array(['Carrefour', 'Auchan', 'Leclerc', 'Casino', 'Leader Price', 'Picard',
                                            'Cora', 'Monoprix', 'Belle France', 'Netto'], dtype=object),
                         'tickvals': array(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'], dtype=object),
                         'title': 'Food Brand',
                         'titlefont': {'color': '#7f7f7f', 'family': 'Courier New, monospace', 'size': 18}},
               'yaxis': {'title': 'Ratio (in %)',
                         'titlefont': {'color': '#7f7f7f', 'family': 'Courier New, monospace', 'size': 18}}}
})

In [76]:
top10_brand_names = ratio.index[:9]

We observe for example that for almost all the biggest food brands present in France the prominent nutrition category is "junk" food. We don't eat that well in France... 

The two biggest exceptions are 'Danone' and 'Picard'. They mainly sell products from milk and frozen food which could be excpected to be healthier than the products sold by Fleury Michon (mainly transformed products from meat).  

Interestingly the 'Bio Village' brand solds a lot of 'junk' food... Even more than 'neutral' food.

### Category Distribution in the 10 biggest food brands in France. 

In [77]:
bra_cat_df = bra_cat_nut_df[['brands', 'pnns_groups_1']]
bra_cat_dummies_df = bra_cat_nut_dummies_df.iloc[:,[0,6,7,8,9,10,11,12,13,14,15]]

In [78]:
# here we count the category occurences for each brand
brand_dummies_count = bra_cat_dummies_df.groupby('brands').sum()

# we add the column total (with the total number of products sold for each brand) to further extract the ratio of each grade instead of the raw count.
brand_count = pd.DataFrame(data=brand_dummies_count.sum(axis=1), columns=['total'])
brand_dummies_count = brand_dummies_count.join(brand_count)
brand_dummies_count.head(3)

,Beverages,Cereals And Potatoes,Composite Foods,Fat And Sauces,Fish Meat Eggs,Fruits And Vegetables,Milk And Dairy Products,Salty Snacks,Sugary Snacks,Unknown,total
brands,,,,,,,,,,,
123 bio,38.0,60.0,28.0,38.0,28.0,64.0,43.0,15.0,72.0,222.0,608.0
7Up,29.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,37.0
A l'olivier,0.0,1.0,2.0,41.0,1.0,0.0,0.0,0.0,2.0,67.0,114.0


In [79]:
# we convert the counts into ratios. (ex: ratio for 'a' = count for 'a' / total number of products)
ratio = brand_dummies_count.copy().reset_index()
ratio = ratio[ratio['brands'].isin(top10_brand_names)].set_index('brands')
ratio.iloc[:,[0,1,2,3,4,5,6,7,8,9]] = brand_dummies_count.iloc[:,np.arange(9)].div(brand_dummies_count['total'].values,axis=0)*100
ratio_to_plot = ratio.reset_index().drop(['total', 'Unknown'],1)
ratio_to_plot.head(3)

,brands,Beverages,Cereals And Potatoes,Composite Foods,Fat And Sauces,Fish Meat Eggs,Fruits And Vegetables,Milk And Dairy Products,Salty Snacks,Sugary Snacks
0,Auchan,5.265011,6.374362,8.681106,4.824793,8.188061,6.691319,8.874802,2.201092,7.941539
1,Belle France,6.370795,7.229778,11.739442,5.941303,9.949893,9.305655,13.815319,4.509664,15.175376
2,Carrefour,4.881403,8.078377,8.525266,4.331385,8.765899,7.356480,7.751805,2.543829,10.244070


In [80]:
data = []
colors = sns.color_palette("tab10", ratio_to_plot.brands.count()).as_hex()

for index, nutri in enumerate(ratio_to_plot.columns[1:]):
    data.append(go.Bar(x=ratio_to_plot['brands'],
                       y=ratio_to_plot[ratio_to_plot.columns[index+1]],
                       name = ratio_to_plot.columns[index+1],
                       marker={'color': colors[index]}))

layout = go.Layout(
    title='Food Category distribution within the 10 biggest food brands in France.',
    xaxis=go.layout.XAxis(
        title='Food Brand',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        ),
        tickvals=np.arange(10).astype(str),
        ticktext=ratio_to_plot['brands'],
    ),
    yaxis=dict(
        title='Ratio (in %)',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    ),
)

go.FigureWidget(data=data, layout=layout)

FigureWidget({
    'data': [{'marker': {'color': '#1f77b4'},
              'name': 'Beverages',
              'type': 'bar',
              'uid': 'c771c73a-fc96-11e8-a01e-ac220bb8d9d3',
              'x': array(['Auchan', 'Belle France', 'Carrefour', 'Casino', 'Cora', 'Leader Price',
                          'Leclerc', 'Monoprix', 'Picard'], dtype=object),
              'y': array([5.26501145, 6.37079456, 4.88140254, 6.23171445, 6.79146713, 5.77478345,
                          6.94073679, 5.84795322, 0.53956835])},
             {'marker': {'color': '#ff7f0e'},
              'name': 'Cereals And Potatoes',
              'type': 'bar',
              'uid': 'c771c73b-fc96-11e8-b7f0-ac220bb8d9d3',
              'x': array(['Auchan', 'Belle France', 'Carrefour', 'Casino', 'Cora', 'Leader Price',
                          'Leclerc', 'Monoprix', 'Picard'], dtype=object),
              'y': array([6.37436168, 7.2297781 , 8.07837745, 7.05090696, 4.65824989, 6.2560154 ,
                          7.23438334, 8.87825625, 2.63788969])},
             {'marker': {'color': '#2ca02c'},
              'name': 'Composite Foods',
              'type': 'bar',
              'uid': 'c771c73c-fc96-11e8-b299-ac220bb8d9d3',
              'x': array(['Auchan', 'Belle France', 'Carrefour', 'Casino', 'Cora', 'Leader Price',
                          'Leclerc', 'Monoprix', 'Picard'], dtype=object),
              'y': array([ 8.68110583, 11.73944166,  8.52526641,  9.21591574,  7.79277318,
                          11.00417068,  8.6225307 , 11.48325359, 37.17026379])},
             {'marker': {'color': '#d62728'},
              'name': 'Fat And Sauces',
              'type': 'bar',
              'uid': 'c771c73d-fc96-11e8-957f-ac220bb8d9d3',
              'x': array(['Auchan', 'Belle France', 'Carrefour', 'Casino', 'Cora', 'Leader Price',
                          'Leclerc', 'Monoprix', 'Picard'], dtype=object),
              'y': array([4.8247931 , 5.94130279, 4.33138536, 3.83265067, 4.30996953, 4.20275906,
                          2.16230646, 4.73152578, 1.73860911])},
             {'marker': {'color': '#9467bd'},
              'name': 'Fish Meat Eggs',
              'type': 'bar',
              'uid': 'c771c73e-fc96-11e8-8329-ac220bb8d9d3',
              'x': array(['Auchan', 'Belle France', 'Carrefour', 'Casino', 'Cora', 'Leader Price',
                          'Leclerc', 'Monoprix', 'Picard'], dtype=object),
              'y': array([8.18806128, 9.94989263, 8.76589893, 6.49502633, 8.6634741 , 8.02053256,
                          8.64922584, 9.94152047, 9.47242206])},
             {'marker': {'color': '#8c564b'},
              'name': 'Fruits And Vegetables',
              'type': 'bar',
              'uid': 'c771c73f-fc96-11e8-bb8c-ac220bb8d9d3',
              'x': array(['Auchan', 'Belle France', 'Carrefour', 'Casino', 'Cora', 'Leader Price',
                          'Leclerc', 'Monoprix', 'Picard'], dtype=object),
              'y': array([ 6.69131889,  9.30565497,  7.35647989,  6.14394383,  6.70439704,
                           6.12768688,  5.12546716,  7.92131845, 10.07194245])},
             {'marker': {'color': '#e377c2'},
              'name': 'Milk And Dairy Products',
              'type': 'bar',
              'uid': 'c771c740-fc96-11e8-bf93-ac220bb8d9d3',
              'x': array(['Auchan', 'Belle France', 'Carrefour', 'Casino', 'Cora', 'Leader Price',
                          'Leclerc', 'Monoprix', 'Picard'], dtype=object),
              'y': array([ 8.8748019 , 13.81531854,  7.75180474,  7.66530135,  6.26904658,
                          10.13795316,  7.02082221,  9.99468368,  8.93285372])},
             {'marker': {'color': '#7f7f7f'},
              'name': 'Salty Snacks',
              'type': 'bar',
              'uid': 'c771c741-fc96-11e8-8c40-ac220bb8d9d3',
              'x': array(['Auchan', 'Belle France', 'Carrefour', 'Casino', 'Cora', 'Leader Price',
                          'Leclerc', 'Monoprix', 'Picard'], dtype=object),
 

In [81]:
data = []
colors = sns.color_palette("tab10", len(ratio_to_plot.columns[1:])).as_hex()

for index, nutri in enumerate(ratio_to_plot.columns[1:]):
    data.append(go.Scatter(x=ratio_to_plot['brands'],
                       y=ratio_to_plot[ratio_to_plot.columns[index+1]],
                       name = ratio_to_plot.columns[index+1],
                       marker={'color': colors[index]}
                          )
               )

In [82]:
fig = tools.make_subplots(rows=3, cols=3, subplot_titles=tuple(ratio_to_plot.columns[1:]))

for index,trace in enumerate(data):
    (r, c) = divmod(index, 3)
    fig.append_trace(trace, r+1, c+1)

fig['layout']['yaxis1'].update(title='Ratio (in %)')
fig['layout']['yaxis4'].update(title='Ratio (in %)')
fig['layout']['yaxis7'].update(title='Ratio (in %)')

fig['layout'].update(title='Food Category distribution within the 10 biggest food brands in France.', height=900, width=1000)

py.iplot(fig)

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]  [ (1,3) x3,y3 ]
[ (2,1) x4,y4 ]  [ (2,2) x5,y5 ]  [ (2,3) x6,y6 ]
[ (3,1) x7,y7 ]  [ (3,2) x8,y8 ]  [ (3,3) x9,y9 ]



## Nutrition grade repartition per category

In [83]:
cat_nut_df = bra_cat_nut_df[['pnns_groups_1', 'nutrition_grade_fr']].dropna()

In [84]:
cat_nut_df = pd.get_dummies(cat_nut_df.set_index('pnns_groups_1')).rename(columns=
                              {'nutrition_grade_fr_a':'a',
                               'nutrition_grade_fr_b':'b',
                               'nutrition_grade_fr_c':'c',
                               'nutrition_grade_fr_d':'d',
                               'nutrition_grade_fr_e':'e'})

In [85]:
count = cat_nut_df.groupby('pnns_groups_1').sum()
count['total'] = count.apply(lambda x: x['a']+x['b']+x['c']+x['d']+x['e'], axis=1) 
count.head(3)

,a,b,c,d,e,total
pnns_groups_1,,,,,,
Beverages,71.0,854.0,1812.0,1624.0,3754.0,8115.0
Cereals And Potatoes,3839.0,1630.0,1616.0,1045.0,101.0,8231.0
Composite Foods,1798.0,3407.0,3404.0,2073.0,185.0,10867.0


In [86]:
ratio = count.copy()
ratio[['a','b','c','d','e']] = count[['a','b','c','d','e']].div(count['total'].values,axis=0)*100
ratio.head(3)

,a,b,c,d,e,total
pnns_groups_1,,,,,,
Beverages,0.874923,10.523722,22.329020,20.012323,46.260012,8115.0
Cereals And Potatoes,46.640748,19.803183,19.633094,12.695906,1.227068,8231.0
Composite Foods,16.545505,31.351799,31.324193,19.076102,1.702402,10867.0


In [87]:
ratio_to_plot = ratio.reset_index().drop('total',1).sort_values(by=['a'],ascending=0).reset_index().drop('index',1)

In [88]:
data = []
colors = sns.color_palette("RdBu_r", 5).as_hex()

for index, nutri in enumerate(ratio_to_plot.columns[1:]):
    data.append(go.Bar(x=ratio_to_plot['pnns_groups_1'],
                       y=ratio_to_plot[ratio_to_plot.columns[index+1]],
                       name = ratio_to_plot.columns[index+1],
                       marker={'color': colors[index]}))

layout = go.Layout(
    title='Health profile for each food category.',
    xaxis=go.layout.XAxis(
        title='Food Category',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        ),
        tickangle = 20,
        tickvals=np.arange(10).astype(str),
        ticktext=ratio_to_plot['pnns_groups_1'],
    ),
    yaxis=dict(
        title='Ratio (in %)',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    ),
)

go.FigureWidget(data=data, layout=layout)

FigureWidget({
    'data': [{'marker': {'color': '#3783bb'},
              'name': 'a',
              'type': 'bar',
              'uid': 'cdcf6398-fc96-11e8-b182-ac220bb8d9d3',
              'x': array(['Fruits And Vegetables', 'Cereals And Potatoes', 'Composite Foods',
                          'Fish Meat Eggs', 'Milk And Dairy Products', 'Unknown',
                          'Fat And Sauces', 'Salty Snacks', 'Beverages', 'Sugary Snacks'],
                         dtype=object),
              'y': array([64.80286738, 46.64074839, 16.54550474,  9.98759305,  7.13354766,
                           7.12445849,  2.3298764 ,  1.88318777,  0.87492298,  0.52667848])},
             {'marker': {'color': '#a7d0e4'},
              'name': 'b',
              'type': 'bar',
              'uid': 'cdcf6399-fc96-11e8-bf45-ac220bb8d9d3',
              'x': array(['Fruits And Vegetables', 'Cereals And Potatoes', 'Composite Foods',
                          'Fish Meat Eggs', 'Milk And Dairy Products', 'Unknown',
                          'Fat And Sauces', 'Salty Snacks', 'Beverages', 'Sugary Snacks'],
                         dtype=object),
              'y': array([17.82078853, 19.80318309, 31.35179902, 11.82795699, 20.14075941,
                          16.02271397,  4.48927404,  3.00218341, 10.5237215 ,  2.32329199])},
             {'marker': {'color': '#f7f6f6'},
              'name': 'c',
              'type': 'bar',
              'uid': 'cdcf639a-fc96-11e8-96d2-ac220bb8d9d3',
              'x': array(['Fruits And Vegetables', 'Cereals And Potatoes', 'Composite Foods',
                          'Fish Meat Eggs', 'Milk And Dairy Products', 'Unknown',
                          'Fat And Sauces', 'Salty Snacks', 'Beverages', 'Sugary Snacks'],
                         dtype=object),
              'y': array([14.89605735, 19.6330944 , 31.32419251, 22.31182796, 29.68111912,
                          19.17808219, 22.16223895, 25.73689956, 22.32902033, 10.87812562])},
             {'marker': {'color': '#f7b799'},
              'name': 'd',
              'type': 'bar',
              'uid': 'cdcf639b-fc96-11e8-b5b3-ac220bb8d9d3',
              'x': array(['Fruits And Vegetables', 'Cereals And Potatoes', 'Composite Foods',
                          'Fish Meat Eggs', 'Milk And Dairy Products', 'Unknown',
                          'Fat And Sauces', 'Salty Snacks', 'Beverages', 'Sugary Snacks'],
                         dtype=object),
              'y': array([ 2.25089606, 12.69590572, 19.07610196, 33.808933  , 39.25623425,
                          26.20302072, 48.41596818, 54.53056769, 20.01232286, 37.23666076])},
             {'marker': {'color': '#c94741'},
              'name': 'e',
              'type': 'bar',
              'uid': 'cdcf639c-fc96-11e8-aa74-ac220bb8d9d3',
              'x': array(['Fruits And Vegetables', 'Cereals And Potatoes', 'Composite Foods',
                          'Fish Meat Eggs', 'Milk And Dairy Products', 'Unknown',
                          'Fat And Sauces', 'Salty Snacks', 'Beverages', 'Sugary Snacks'],
                         dtype=object),
              'y': array([ 0.22939068,  1.2270684 ,  1.70240177, 22.063689  ,  3.78833956,
                          31.47172462, 22.60264242, 14.84716157, 46.26001232, 49.03524316])}],
    'layout': {'title': 'Health profile for each food category.',
               'xaxis': {'tickangle': 20,
                         'ticktext': array(['Fruits And Vegetables', 'Cereals And Potatoes', 'Composite Foods',
                                            'Fish Meat Eggs', 'Milk And Dairy Products', 'Unknown',
                                            'Fat And Sauces', 'Salty Snacks', 'Beverages', 'Sugary Snacks'],
                                           dtype=object),
                         'tickvals': array(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'], dtype=object),
                         'title': 'Food Category',
                         'titlefont': {'color': '#7f7f7f', 'f